## Imports

In [58]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten

In [59]:
df = pd.read_csv('LA_map_popup_pin - ktown.csv')

In [60]:
df.columns

Index(['full_address', 'developers', 'total_units',
       'affordable_or_extremely_low_income_units', 'description',
       'story_link'],
      dtype='object')

## Image URL collector

In [61]:
### Drop last two rows of a df. In case there are sums that need clipped
df.drop(df.tail(2).index, inplace=True)

In [62]:
df['image_links'] = ''
links = []
for url in df.story_link:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    images = soup.find_all('img')
    counter = 0
    for image in images:
        counter += 1
        if 'jpg' in image['src']:
            links.append(image['src'])
            if counter == len(images):
                links.append('no image found')
            break

df['image_links'] = links

## Article link formatter

In [63]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/la/2022/06/23...
1    <a href="https://therealdeal.com/la/2022/06/06...
2    <a href="https://therealdeal.com/la/2022/04/27...
3    <a href="https://therealdeal.com/la/2022/04/22...
4    <a href="https://therealdeal.com/la/2022/03/23...
5    <a href="https://therealdeal.com/la/2021/12/16...
6    <a href="https://therealdeal.com/la/2021/11/29...
7    <a href="https://therealdeal.com/la/2021/10/26...
8    <a href="https://therealdeal.com/la/2021/11/03...
9    <a href="https://therealdeal.com/la/2021/09/22...
Name: description_link, dtype: object

In [64]:
df.story_link[0]

'https://therealdeal.com/la/2022/06/23/goodbye-dragon-hello-housing-yet-another-apartment-complex-planned-for-koreatown/'

## Google Maps API Geolocater Setup

In [65]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [66]:
df['geo_address'] = df['full_address'] + ', Los Angeles, CA'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## HTML popup formatter

In [67]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developer = df['developers'].iloc[i]
    description = df['description_link'].iloc[i]
    image = df['image_links'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>
    <strong>{}'''.format(description) + '''</strong></html>
    '''
    return html


## Map Maker

In [68]:
df.description_link

0    <a href="https://therealdeal.com/la/2022/06/23...
1    <a href="https://therealdeal.com/la/2022/06/06...
2    <a href="https://therealdeal.com/la/2022/04/27...
3    <a href="https://therealdeal.com/la/2022/04/22...
4    <a href="https://therealdeal.com/la/2022/03/23...
5    <a href="https://therealdeal.com/la/2021/12/16...
6    <a href="https://therealdeal.com/la/2021/11/29...
7    <a href="https://therealdeal.com/la/2021/10/26...
8    <a href="https://therealdeal.com/la/2021/11/03...
9    <a href="https://therealdeal.com/la/2021/09/22...
Name: description_link, dtype: object

In [69]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'New Development in Koreatown')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [70]:
m.save('index.html')